In [ ]:
import gym
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from collections import deque
import random

# Create the environment
env = gym.make('CartPole-v1')

# Define the DQN agent
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95  # Discount factor
        self.epsilon = 1.0  # Exploration rate
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        self.learning_rate = 0.001
        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state, verbose=0)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma * np.amax(self.model.predict(next_state, verbose=0)[0]))
            target_f = self.model.predict(state, verbose=0)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# Initialize the agent
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
agent = DQNAgent(state_size, action_size)

# Train the agent
episodes = 1000
batch_size = 32

for e in range(episodes):
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    for time in range(500):
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        reward = reward if not done else -10
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        if done:
            print(f"Episode: {e}/{episodes}, Score: {time}, Epsilon: {agent.epsilon:.2}")
            break
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)

## Explanation of the DQN Implementation

1. **Environment Setup**: The `CartPole-v1` environment from OpenAI Gym is used to simulate the inverted pendulum problem. The goal is to balance a pole on a cart by applying forces to the cart.

2. **DQN Agent**:
   - The agent is defined with a neural network model that approximates the Q-function.
   - The model has three layers: two hidden layers with 24 neurons each and ReLU activation, and an output layer with linear activation.
   - The agent uses an epsilon-greedy policy for exploration and exploitation.

3. **Experience Replay**:
   - The agent stores past experiences in a replay buffer.
   - During training, random batches are sampled from the buffer to update the Q-values, improving stability and efficiency.

4. **Training Loop**:
   - For each episode, the agent interacts with the environment, selects actions, and stores experiences.
   - The agent trains on batches of experiences from the replay buffer to update its Q-function.
   - The exploration rate (`epsilon`) decays over time to shift from exploration to exploitation.

5. **Performance Metrics**:
   - The score for each episode is printed, along with the current exploration rate.
   - The training continues for 1000 episodes or until the agent achieves satisfactory performance.